In [16]:
import pandas as pd

df = pd.read_csv('archive_raw/olympic_results.csv')

In [17]:
df['slug_game'].unique()

array(['beijing-2022', 'tokyo-2020', 'pyeongchang-2018', 'rio-2016',
       'sochi-2014', 'london-2012', 'vancouver-2010', 'beijing-2008',
       'turin-2006', 'athens-2004', 'salt-lake-city-2002', 'sydney-2000',
       'nagano-1998', 'atlanta-1996', 'lillehammer-1994',
       'barcelona-1992', 'albertville-1992', 'seoul-1988', 'calgary-1988',
       'los-angeles-1984', 'sarajevo-1984', 'moscow-1980',
       'lake-placid-1980', 'montreal-1976', 'innsbruck-1976',
       'munich-1972', 'sapporo-1972', 'mexico-city-1968', 'grenoble-1968',
       'tokyo-1964', 'innsbruck-1964', 'rome-1960', 'squaw-valley-1960',
       'melbourne-1956', 'cortina-d-ampezzo-1956', 'helsinki-1952',
       'oslo-1952', 'london-1948', 'st-moritz-1948', 'berlin-1936',
       'garmisch-partenkirchen-1936', 'los-angeles-1932',
       'lake-placid-1932', 'amsterdam-1928', 'st-moritz-1928',
       'paris-1924', 'chamonix-1924', 'antwerp-1920', 'stockholm-1912',
       'london-1908', 'st-louis-1904', 'paris-1900', 'at

In [18]:
# éditions d'été de 1948 à 2020
game_slugs = [
    'tokyo-2020',  'rio-2016',
    'london-2012', 'beijing-2008',
    'athens-2004', 'sydney-2000', 'atlanta-1996',
    'barcelona-1992', 'seoul-1988',
    'los-angeles-1984',  'moscow-1980', 'montreal-1976',
    'munich-1972', 'mexico-city-1968',
    'tokyo-1964', 'rome-1960',
    'melbourne-1956', 'helsinki-1952','london-1948'
]

df = df[df['slug_game'].apply(lambda x: x in game_slugs)]

df['slug_game'].unique()

array(['tokyo-2020', 'rio-2016', 'london-2012', 'beijing-2008',
       'athens-2004', 'sydney-2000', 'atlanta-1996', 'barcelona-1992',
       'seoul-1988', 'los-angeles-1984', 'moscow-1980', 'montreal-1976',
       'munich-1972', 'mexico-city-1968', 'tokyo-1964', 'rome-1960',
       'melbourne-1956', 'helsinki-1952', 'london-1948'], dtype=object)

In [19]:
# sports continus de 1948 à 1988
sports_summer_before = ['Shooting', 'Diving', 'Canoe Sprint', 'Cycling Road', 'Football',
                 'Boxing', 'Basketball', 'Cycling Track', 'Fencing',
                 'Water Polo', 'Wrestling', 'Artistic Gymnastics',
                 'Weightlifting', 'Modern Pentathlon', 'Hockey'
                 'Athletics', 'Swimming', 'Sailing', 'Rowing']

# éditions de 1948 à 1988
editions_to_1988 = ['seoul-1988',
    'los-angeles-1984',  'moscow-1980', 'montreal-1976',
    'munich-1972', 'mexico-city-1968',
    'tokyo-1964', 'rome-1960',
    'melbourne-1956', 'helsinki-1952','london-1948'
]


df_summer_before_1988 = df[df['discipline_title'].apply(lambda x: x in sports_summer_before)]
df_summer_before_1988 = df_summer_before_1988[df_summer_before_1988['slug_game'].apply(lambda x: x in editions_to_1988)]


In [20]:
# sports continus de 1992 à 2020
sports_summer_after = ['Shooting', 'Diving', 'Canoe Sprint', 'Cycling Road', 'Football',
                 'Boxing', 'Basketball', 'Cycling Track', 'Fencing', 'Table Tennis', 'Badminton',
                 'Water Polo', 'Wrestling', 'Artistic Gymnastics', 'Canoe Slalom', 'Rhythmic Gymnastics',
                 'Weightlifting', 'Modern Pentathlon', 'Hockey', 'Volleyball', 'Artistic Swimming', 
                 'Athletics', 'Swimming', 'Sailing', 'Rowing', 'Tennis', 'Equestrian', 'Archery', 'Handball', 'Judo']

# éditions de 1992 à 2020
editions_after_1988 = ['tokyo-2020',  'rio-2016',
    'london-2012', 'beijing-2008',
    'athens-2004', 'sydney-2000', 'atlanta-1996',
    'barcelona-1992']


df_summer_after_1988 = df[df['discipline_title'].apply(lambda x: x in sports_summer_after)]
df_summer_after_1988 = df_summer_after_1988[df_summer_after_1988['slug_game'].apply(lambda x: x in editions_after_1988)]

In [21]:
def extract_10_first(df):
    df['rank_position'] = pd.to_numeric(df['rank_position'], errors='coerce')
    df_cl = df.dropna(subset=['rank_position'])
    df_cl['rank_position'] = df_cl['rank_position'].astype(int)
    df_cl = df_cl[df_cl['rank_position'] <= 10]
    return df_cl



filtered_df = extract_10_first(df_summer_after_1988)
df_after = filtered_df.groupby(by=['country_name', 'slug_game']).count()



filtered_df_before = extract_10_first(df_summer_before_1988)
df_before = filtered_df_before.groupby(by=['country_name', 'slug_game']).count()


C:\Users\Julien\AppData\Local\Temp\ipykernel_13652\3581285298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cl['rank_position'] = df_cl['rank_position'].astype(int)
C:\Users\Julien\AppData\Local\Temp\ipykernel_13652\3581285298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cl['rank_position'] = df_cl['rank_position'].astype(int)


In [ ]:
df_clean_after = df_after['rank_position']
df_clean_after.to_csv('data_rank_after.csv')


df_clean_before = df_before['rank_position']
df_clean_before.to_csv('data_rank_before.csv')


In [23]:
#df_summer_after_1988[ (df_summer_after_1988['country_name'] == 'Algeria') & (df_summer_after_1988['slug_game'] == 'athens-2004')
#                    & (df_summer_after_1988['rank_position'] <= 10)]

In [ ]:
data = pd.read_csv('data_rank_after.csv')
df = pd.DataFrame(data)

df['year'] = df['slug_game'].str.extract(r'(\d{4})')

df_pivot = df.pivot_table(index='country_name', columns='year', values='rank_position', aggfunc='first')

df_pivot.reset_index(inplace=True)
df_pivot.fillna(0, inplace=True)

df_pivot = df_pivot.transpose()

df_pivot = df_pivot.iloc[0:].reset_index(drop=True)

df_pivot.columns = df_pivot.iloc[0]
df_pivot = df_pivot.drop(0).reset_index(drop=True)

df_pivot.to_csv('second_part.csv')


KeyError: 'slug_game'

In [ ]:
data = pd.read_csv('data_rank_before.csv')
df = pd.DataFrame(data)

df['year'] = df['slug_game'].str.extract(r'(\d{4})')

df_pivot = df.pivot_table(index='country_name', columns='year', values='rank_position', aggfunc='first')

df_pivot.reset_index(inplace=True)
df_pivot.fillna(0, inplace=True)

df_pivot = df_pivot.transpose()


df_pivot = df_pivot.iloc[0:].reset_index(drop=True)

df_pivot.columns = df_pivot.iloc[0]
df_pivot = df_pivot.drop(0).reset_index(drop=True)


df_pivot.to_csv('first_part.csv')

In [ ]:
first_part = pd.read_csv('first_part.csv')
second_part = pd.read_csv('second_part.csv')


In [ ]:
# expot list of countries 
country_list = first_part.columns[1:].tolist() + second_part.columns[1:].tolist()
country_list = list(set(country_list))
country_list.sort()
with open('country_list.txt', 'w') as f:
    for country in country_list:
        f.write(f"{country}\n")

In [ ]:
second_part

,Unnamed: 0,Albania,Algeria,American Samoa,Andorra,Angola,Argentina,Armenia,Aruba,Australia,...,United Arab Emirates,United Republic of Tanzania,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,"Virgin Islands, British",Zambia,Zimbabwe
0,0,1.0,14.0,2.0,0.0,1.0,12.0,0.0,0.0,78.0,...,0.0,5.0,226.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0
1,1,0.0,11.0,0.0,0.0,1.0,18.0,17.0,0.0,105.0,...,0.0,0.0,220.0,1.0,16.0,6.0,0.0,0.0,4.0,1.0
2,2,1.0,11.0,0.0,0.0,1.0,20.0,7.0,0.0,144.0,...,1.0,0.0,204.0,2.0,18.0,9.0,0.0,0.0,1.0,1.0
3,3,0.0,11.0,0.0,0.0,1.0,17.0,9.0,0.0,128.0,...,2.0,1.0,196.0,2.0,16.0,9.0,1.0,0.0,1.0,4.0
4,4,2.0,11.0,0.0,0.0,0.0,12.0,11.0,0.0,123.0,...,1.0,1.0,200.0,0.0,16.0,12.0,2.0,0.0,0.0,6.0
5,5,3.0,6.0,0.0,1.0,2.0,16.0,7.0,0.0,107.0,...,1.0,0.0,213.0,2.0,19.0,14.0,5.0,0.0,0.0,3.0
6,6,2.0,13.0,0.0,0.0,2.0,19.0,13.0,1.0,122.0,...,3.0,1.0,210.0,1.0,25.0,13.0,5.0,0.0,1.0,1.0
7,7,2.0,9.0,0.0,0.0,1.0,17.0,9.0,0.0,115.0,...,0.0,1.0,236.0,1.0,29.0,7.0,2.0,1.0,4.0,0.0
